In [13]:
import heapq  # 导入堆队列模块，用于实现优先队列

In [14]:
class Node:
    def __init__(self, state, parent=None, path_cost=0):
        self.state = state       # 节点状态
        self.parent = parent     # 父节点
        self.path_cost = path_cost  # 路径成本

    def __lt__(self, other):
        # 定义节点在优先队列中的比较基于路径成本
        return self.path_cost < other.path_cost

In [15]:
#交换列表中的两个元素位置
def swap(state, i, j):
    state[i], state[j] = state[j], state[i]
    return state

def possible_moves(state):
    # 返回八数码问题中，空白格（0）可以移动的所有位置
    # 取决于空白格当前的位置
    index = state.index(0)  # 找到空白格的当前位置
    moves = []
    # 判断空白格可以移动的方向，并添加到移动列表
    if index % 3 != 0:  # 空白格不在第一列，可以左移
        moves.append(index - 1)
    if index % 3 != 2:  # 空白格不在最后一列，可以右移
        moves.append(index + 1)
    if index >= 3:  # 空白格不在第一行，可以上移
        moves.append(index - 3)
    if index < 6:  # 空白格不在最后一行，可以下移
        moves.append(index + 3)
    return moves


def print_solution(node):
    # 从解决方案节点逆向遍历到初始节点，打印所有状态
    path = []
    while node:
        path.append(node.state)  # 将当前节点的状态添加到路径列表
        node = node.parent  # 移动到父节点
    for state in path[::-1]:  # 反向遍历路径列表，打印从初始到目标的状态
        print(state)

In [ ]:
# 通过对当前node应用所有可能的移动操作，扩展节点，返回所有可能的后继状态
def expand(node):
    successors = []
    # 空白格位置    
    index = node.state.index(0)
    # 获取所有可能的移动
    moves = possible_moves(node.state)

    # 对当前node应用所有可能的移动操作
    for move in moves:
        # 复制当前状态以创建新状态
        new_state = node.state[:]  

        # 执行移动
        swap(new_state, index, move)  

        # 创建新节点
        new_node = Node(new_state, node, path_cost=node.path_cost + 1)  
        
        
        successors.append(new_node)  # 添加到后继列表
    return successors

In [16]:
# 实现A*搜索算法，寻找从初始状态到目标状态的路径
def a_star_search(initial, goal):
    # 创建初始节点
    initial_node = Node(initial)  

    # 将初始节点加入前沿队列
    frontier = []  # 前沿队列，用于存储待探索的节点
    heapq.heappush(frontier, initial_node)  
    
    explored = set()  # 已探索集合
    while frontier:
        # 取出并删除前沿队列中路径成本最小的节点
        node = heapq.heappop(frontier)  

        # 检查当前节点的状态是否是目标状态
        if node.state == goal:  
            return node
        
        # 将当前状态添加到已探索集
        explored.add(tuple(node.state))  
        
        # iterate all successors of current node
        for successor in expand(node):  # 生成所有后继节点
            # 将未探索的后继节点加入前沿队列
            if tuple(successor.state) not in explored:
                heapq.heappush(frontier, successor)  
    return None

In [17]:
# 八数码问题的初始和目标状态
initial_state = [1, 2, 3, 
                 4, 5, 6, 
                 0, 7, 8]
goal_state = [1, 2, 3, 
              4, 5, 6, 
              7, 8, 0]

# 运行A*搜索
solution = a_star_search(initial_state, goal_state)

# 如果找到解决方案，打印所有中间状态
if solution:
    print_solution(solution)
else:
    print("No solution found.")

[1, 2, 3, 4, 5, 6, 0, 7, 8]
[1, 2, 3, 4, 5, 6, 7, 0, 8]
[1, 2, 3, 4, 5, 6, 7, 8, 0]


In [ ]:
import heapq
import math

ALGORITHM = 1

Puzzle_Number = 8
NUM_ROW = int(math.sqrt(Puzzle_Number + 1))

class Node:
    def __init__(self, state, goal, parent=None, action=None, path_cost=0):
        self.state = state  # 当前节点的状态，例如八数码问题中的一个具体排列
        self.parent = parent  # 指向父节点的引用，通过这个可以追踪到起始状态
        self.action = action  # 到达当前状态所执行的操作，例如“向左移动空白格”
        self.path_cost = path_cost  # 从初始状态到当前节点的路径成本 g(n)

        # 启发式成本 h(n)
        if ALGORITHM == 1:
            self.heuristic_cost = self.UniformCostSearch()
        elif ALGORITHM == 2:
            self.heuristic_cost = self.MisplacedTileHeuristic(goal)
        else:
            self.heuristic_cost = self.ManhattanDistance(goal)
        
        # 总成本f(n) = 路径成本g(n) + 启发式成本h(n)
        self.total_cost = self.path_cost + self.heuristic_cost  

    # 定义两个节点间的小于比较，基于它们的总成本 f(n)。
    # 这个比较函数在优先队列中用于决定节点的弹出顺序。
    def __lt__(self, other):
        return self.total_cost < other.total_cost
    
    # Uniform Cost Search
    def UniformCostSearch(self):
        return 0
    
    # Manhattan Distance heuristic
    def ManhattanDistance(self, goal):
        state = self.state
        distance = 0
        # iterate each tiles, and count distance from current position to goal position
        for i in range(len(state)):
            if state[i] != 0:  # 忽略空白格
                target_index = goal.index(state[i])
                # calculate horizontal distance + vertical distance
                distance += abs(target_index % NUM_ROW - i % NUM_ROW) + abs(target_index // NUM_ROW - i // NUM_ROW)
        
        return distance
    
    # Misplaced Tile Heuristic
    def MisplacedTileHeuristic(self, goal):
        state = self.state
        misplaced_count = 0

        # count how many tiles are not in goal positions
        for i in range(len(state)):
            if state[i] != 0 and state[i] != goal[i]:  # 不计算空白格（0）的位置
                misplaced_count += 1
        
        return misplaced_count


#交换列表中的两个元素位置
def swap(state, i, j):
    state[i], state[j] = state[j], state[i]
    return state


# 返回八数码问题中，空白格0可以移动的所有位置
def possible_moves(state):
    moves = []
    # 找到空白格的当前位置
    index = state.index(0)  

    # 判断空白格可以移动的方向，并添加到移动列表
    # 空白格不在第一列，可以左移
    if index % NUM_ROW != 0:  
        moves.append(index - 1)
    # 空白格不在最后一列，可以右移
    if index % NUM_ROW != NUM_ROW - 1:  
        moves.append(index + 1)
    # 空白格不在第一行，可以上移
    if index >= NUM_ROW:  
        moves.append(index - NUM_ROW)
    # 空白格不在最后一行，可以下移
    if index < NUM_ROW * (NUM_ROW - 1):  
        moves.append(index + NUM_ROW)

    return moves


def print_solution(node):
    path = []
    while node:
        path.append(node)
        node = node.parent
    number = 1
    for step in path[::-1]:
        state = step.state
        
        print(f"State {number}: ")

        for x in range(NUM_ROW):
            for y in range(NUM_ROW):
                print(f"{state[x * NUM_ROW + y]}", end=' ')
            print(" ")

        print(f"g(n): {step.path_cost}, h(n): {step.heuristic_cost}\n")
        # print(f"State: \n[{state[0]}, {state[1]}, {state[2]}]\n[{state[3]}, {state[4]}, {state[5]}]\n[{state[6]}, {state[7]}, {state[8]}], g(n): {step.path_cost}, h(n): {step.heuristic_cost}")

        number += 1


# 通过对当前node应用所有可能的移动操作，扩展节点，返回所有可能的后继状态
def expand(node, goal):
    successors = []
    # 空白格位置    
    index = node.state.index(0)
    # 获取所有可能的移动
    moves = possible_moves(node.state)

    # 对当前node应用所有可能的移动操作
    for move in moves:
        # 复制当前状态以创建新状态
        new_state = node.state[:]  

        # 执行移动
        swap(new_state, index, move)  

        # 创建新节点
        new_node = Node(new_state, goal, node, path_cost=node.path_cost + 1)  
        
        # 添加到后继列表
        successors.append(new_node)  
    return successors


# 实现A*搜索算法，寻找从初始状态到目标状态的路径
def a_star_search(initial, goal):
    # 创建初始节点
    initial_node = Node(initial, goal)  

    # 将初始节点加入前沿队列
    frontier = []  # 前沿队列，用于存储待探索的节点
    heapq.heappush(frontier, initial_node)  
    
    explored = set()  # 已探索集合
    while frontier:
        # 取出并删除前沿队列中路径成本最小的节点
        node = heapq.heappop(frontier)  

        # 检查当前节点的状态是否是目标状态
        if node.state == goal:  
            return node
        
        # 将当前状态添加到已探索集
        explored.add(tuple(node.state))  
        
        # iterate all successors of current node
        for successor in expand(node, goal):  # 生成所有后继节点
            # 将未探索的后继节点加入前沿队列
            if tuple(successor.state) not in explored:
                heapq.heappush(frontier, successor)  
    return None


# 八数码问题的初始和目标状态
initial_state = [7, 1, 2, 
                 4, 8, 5, 
                 6, 3, 0]
goal_state = [1, 2, 3, 
              4, 5, 6, 
              7, 8, 0]


# Uniform Cost Search
# 运行A*搜索
solution = a_star_search(initial_state, goal_state)

print("Uniform Cost Search: \n")

# 如果找到解决方案，打印所有中间状态及其成本
if solution:
    print_solution(solution)
else:
    print("No solution found.")